In [1]:
from hap import HomeAutoPilot
from hap_util import *

# data prep

In [ ]:
areas =  ['backyard', 'frontyard', 'garage', 'kitchen', 'livingroom']
data_files = [f'../data/{area}.csv' for area in areas]
input_tensors, act_dev_tensors, act_state_tensors, act_timing_tensors, n_features, n_act_state, n_act_dev, unique_entity_ids, unique_event_states = prep_tensors(data_files)

In [2]:
input_tensors = torch.load('input_seq_tensors.pt', weights_only=True)
act_dev_tensors = torch.load('act_dev_tensors.pt', weights_only=True)
act_state_tensors = torch.load('act_state_tensors.pt', weights_only=True)
act_timing_tensors = torch.load('act_timing_tensors.pt', weights_only=True)
device_id_map = torch.load('device_id_map.pt', weights_only=False)
device_state_id_map = torch.load('device_state_map.pt', weights_only=False)

n_act_state = len(device_state_id_map)
n_act_dev = len(device_id_map)

# run inferencing

In [4]:
# re-create the model
pilot = HomeAutoPilot(num_devices = n_act_dev, num_device_actions = n_act_state)
pilot.load_model('pilot_demo.ptmodel')

INFO:hap:Using device: cuda


In [237]:
import random

sample_idx = random.choice(range(len(input_tensors)))
input_tensor = input_tensors[sample_idx]
predicted_timing, predicted_device_idx, predicted_action_idx = pilot.eval(input_tensor)
device_name = device_id_map[predicted_device_idx]
action_name = device_state_id_map[predicted_action_idx]

print(f'prediction: Turn {device_name} {action_name} after {predicted_timing} seconds')

prediction: Turn binary_sensor.frontporch_person_occupancy off after 18.07762336730957 seconds


# train model

In [ ]:
pilot = HomeAutoPilot(num_devices = n_act_dev, num_device_actions = n_act_state)
pilot.train_model(input_tensors, act_dev_tensors, act_state_tensors, act_timing_tensors, model_path = 'pilot_demo.ptmodel')